# Production model
[![prod.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/amplcolab/blob/master/book/prod.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplcolab/blob/master/book/prod.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplcolab/blob/master/book/prod.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/amplcolab/blob/master/book/prod.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/amplcolab/blob/master/book/prod.ipynb)

Description: generic model for production problem

Tags: ampl-only, ampl-book

Notebook author: N/A

Model author: N/A


This notebook provides the implementation of the production problem described in the book *AMPL: A Modeling Language for Mathematical Programming*
by Robert Fourer, David M. Gay, and Brian W. Kernighan.

The following cells download the necessary dependences to use the AMPL API for Python (AMPLPY).

In [ ]:
# Install dependencies
!pip install -q amplpy ampltools

     |████████████████████████████████| 4.1 MB 7.2 MB/s 


In [ ]:
# Google Colab & Kaggle interagration
MODULES=['ampl', 'coin']
from ampltools import cloud_platform_name, ampl_notebook
from amplpy import AMPL, register_magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = ampl_notebook(modules=MODULES) # Install AMPL and use it
register_magics(ampl_object=ampl) # Evaluate %%ampl_eval cells with ampl.eval()

Downloading: https://portal.ampl.com/dl/modules/ampl-module.linux64.tgz
> /content/ampl.linux-intel64/ampl.lic (new)
> /content/ampl.linux-intel64/eodbc.dll (new)
> /content/ampl.linux-intel64/models (new)
> /content/ampl.linux-intel64/ampl (new)
> /content/ampl.linux-intel64/ampl.env (new)
> /content/ampl.linux-intel64/fingerprint (new)
> /content/ampl.linux-intel64/amplxl.dll (new)
> /content/ampl.linux-intel64/amplkey (new)
> /content/ampl.linux-intel64/amplcsv.dll (new)
> /content/ampl.linux-intel64/README (new)
> /content/ampl.linux-intel64/leasefingerprint (new)
> /content/ampl.linux-intel64/docs (new)
Downloading: https://portal.ampl.com/dl/modules/coin-module.linux64.tgz
> /content/ampl.linux-intel64/bonmin (new)
> /content/ampl.linux-intel64/coin-license.txt (new)
> /content/ampl.linux-intel64/couenne (new)
> /content/ampl.linux-intel64/coin-versions.txt (new)
> /content/ampl.linux-intel64/ipopt (new)
> /content/ampl.linux-intel64/cbc (new)
AMPL License:


## Example: production model

It is usual to adopt mathematical notation as a general and concise way of expressing problems based on variables, constraints, and objectives. We can write a compact description of the general form of a production problem, which we call a *model*, using algebraic notation for the objective and the constraints.

### Algebraic formulation

Given:

* $P$, a set of products
* $a_j$ = tons per hour of product $j$, for each $j \in P$
* $b$ = hours available at the mill
* $c_j$ = profit per ton of product $j$, for each $j \in P$
* $u_j$ = maximum tons of product $j$, for each $j \in P$

Define variables:   $X_j$ = tons of product $j$ to be made, for each $j \in P$.

Maximize:
$$\sum \limits_{j \in P} c_j X_j$$

Subject to:
$$\sum \limits_{j \in P} \frac{1}{a_j} X_j \leq b$$

$$0 \leq X_j \leq u_j, \text{ for each }j \in P$$

The model describes an infinite number of related optimization problems. If we provide specific values for data, however, the model becomes a specific problem, or instance of the model, that can be solved. Each different collection of data values defines a different instance.


### Model implementation

The general formulation above can be written with AMPL as follows:

In [ ]:
%%writefile prod.mod
# Sets and parameters
set P;

param a {j in P};
param b;
param c {j in P};
param u {j in P};

# Variables
var X {j in P};

# Objective function
maximize Total_Profit: sum {j in P} c[j] * X[j];

# Time and Limits constraints
subject to Time: sum {j in P} (1/a[j]) * X[j] <= b;

subject to Limit {j in P}: 0 <= X[j] <= u[j];

Writing prod.mod


### Data

Due to the model and data separation, the abstract formulation works for any correct data input we provide to AMPL. A possible instance of the production problem is the following:

In [ ]:
%%writefile prod.dat

set P := bands coils;

param:     a     c     u  :=
  bands   200   25   6000
  coils   140   30   4000 ;

param b := 40;

Writing prod.dat


### Solve the problem

We can load the generated model and data files, and solve them by using a linear solver as CBC. Finally, the solution (values for X) is displayed.

In [ ]:
%%ampl_eval
model prod.mod;
data prod.dat;
option solver cbc;
solve;
display X;

CBC 2.10.5: CBC 2.10.5 optimal, objective 192000
1 iterations

"option abs_boundtol 9.094947017729282e-13;"
or "option rel_boundtol 1.5158245029548803e-16;"
will change deduced dual values.

X [*] :=
bands  6000
coils  1400
;

